<a href="https://colab.research.google.com/github/yyparty/WTFSolidity/blob/main/%E2%80%9Cblip%E2%80%9D-2024-02-26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install GitPython==3.1.32 Pillow==9.5.0 accelerate==0.21.0 basicsr==1.4.2 blendmodes==2022 clean-fid==0.1.35 einops==0.4.1 fastapi==0.94.0 gfpgan==1.3.8 gradio==3.41.2 httpcore==0.15 inflection==0.5.1 jsonmerge==1.8.0 kornia==0.6.7 lark==1.1.2 numpy==1.23.5 omegaconf==2.2.3 open-clip-torch==2.20.0 piexif==1.1.3 psutil==5.9.5 pytorch_lightning==1.9.4 realesrgan==0.3.0 resize-right==0.0.2 safetensors==0.3.1 scikit-image==0.21.0 timm==0.9.2 tomesd==0.1.3 torch torchdiffeq==0.2.3 torchsde==0.2.5 transformers==4.30.2 httpx==0.24.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [7]:
!pip install timm==0.9.2

In [8]:
blip_commit_hash = "48211a1594f1321b00f14c9f7a5b4813144b2fb9"
!git clone https://github.com/salesforce/BLIP.git

Cloning into 'BLIP'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 277 (delta 137), reused 136 (delta 135), pack-reused 112
Receiving objects: 100% (277/277), 7.03 MiB | 23.85 MiB/s, done.
Resolving deltas: 100% (152/152), done.


In [9]:
import os

os.chdir("/content/BLIP")

import torch
import requests
from PIL import Image
from urllib.parse import urlparse
from transformers import (
    BertTokenizer,
    BlipConfig,
    BlipForConditionalGeneration,
    BlipForImageTextRetrieval,
    BlipForQuestionAnswering,
    Blip2Processor,
    Blip2ForConditionalGeneration
)

import sys
def create_fake_fairscale():
    class FakeFairscale:
        def checkpoint_wrapper(self):
            pass

    sys.modules["fairscale.nn.checkpoint.checkpoint_activations"] = FakeFairscale
create_fake_fairscale()
from models.blip import blip_decoder

In [10]:
model_url = (
    "https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth"
)
def load_file_from_url(
    url: str,
    *,
    model_dir: str,
    progress: bool = True,
    file_name: str | None = None,
) -> str:
    """Download a file from `url` into `model_dir`, using the file present if possible.

    Returns the path to the downloaded file.
    """
    os.makedirs(model_dir, exist_ok=True)
    if not file_name:
        parts = urlparse(url)
        file_name = os.path.basename(parts.path)
    cached_file = os.path.abspath(os.path.join(model_dir, file_name))
    if not os.path.exists(cached_file):
        print(f'Downloading: "{url}" to {cached_file}\n')
        from torch.hub import download_url_to_file
        download_url_to_file(url, cached_file, progress=progress)
    return cached_file

model_file = load_file_from_url(model_url, model_dir="/content/models/BLIP", file_name="model_base_capfilt_large.pth")
blip_model = blip_decoder(
    pretrained=model_file,
    image_size=384,
    vit="base",
)
blip_model = blip_model.to(torch.device("cuda"))
blip_model.eval()
print('load success')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

load checkpoint from /content/models/BLIP/model_base_capfilt_large.pth
load success


../compressed.zip has been created.


In [19]:
# @title 默认标题文本
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
img_url = "https://img001.aivideo8.com/ai_video/U62/2023-11-22/img/1700644729154.jpg"


import os

# Specify the directory path
directory_path = "../data2/"
# Get a list of all files in the directory
files = os.listdir(directory_path)

# Filter out only the files with the extensions ".jpg" or ".png"
image_files = [file for file in files if file.endswith(".jpg") or file.endswith(".png")]

# Print the list of image files
for image_file in image_files:
    print(image_file)

    #pil_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
    pil_image = Image.open(directory_path+image_file).convert("RGB")

    blip_image_eval_size = 384
    from torchvision import transforms
    from torchvision.transforms.functional import InterpolationMode

    gpu_image = (
        transforms.Compose(
            [
                transforms.Resize(
                    (blip_image_eval_size, blip_image_eval_size), interpolation=InterpolationMode.BICUBIC
                ),
                transforms.ToTensor(),
                transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
            ]
        )(pil_image)
        .unsqueeze(0)
        .to(torch.device("cuda"))
    )

    caption = blip_model.generate(
        gpu_image,
        sample=False,
        num_beams=1,
        min_length=24,
        max_length=48,
    )
    caption
    # Open a file in write mode
    file_path = directory_path+image_file.replace(".jpg",".txt").replace(".png",".txt")
    with open(file_path, 'w') as file:
        # Write a string to the file
        file.write(caption[0])




33ifashui-512x512.jpg
64ifashui-512x512.jpg
20ifashui-512x512.jpg
100ifashui-512x512.jpg
22ifashui-512x512.jpg
46ifashui-512x512.jpg
109ifashui-512x512.jpg
17ifashui-512x512.jpg
10ifashui-512x512.jpg
122ifashui-512x512.jpg
123ifashui-512x512.jpg
49ifashui-512x512.jpg
132ifashui-512x512.jpg
41ifashui-512x512.jpg
19ifashui-512x512.jpg
124ifashui-512x512.jpg
65ifashui-512x512.jpg
75ifashui-512x512.jpg
39ifashui-512x512.jpg
47ifashui-512x512.jpg
51ifashui-512x512.jpg
34ifashui-512x512.jpg
30ifashui-512x512.jpg
70ifashui-512x512.jpg
15ifashui-512x512.jpg
55ifashui-512x512.jpg
11ifashui-512x512.jpg
121ifashui-512x512.jpg
5ifashui-512x512.jpg
115ifashui-512x512.jpg
71ifashui-512x512.jpg
126ifashui-512x512.jpg
7ifashui-512x512.jpg
113ifashui-512x512.jpg
86ifashui-512x512.jpg
135ifashui-512x512.jpg
13ifashui-512x512.jpg
38ifashui-512x512.jpg
31ifashui-512x512.jpg
67ifashui-512x512.jpg
44ifashui-512x512.jpg
89ifashui-512x512.jpg
63ifashui-512x512.jpg
104ifashui-512x512.jpg
48ifashui-512x512.jpg


In [ ]:
import zipfile
import os

def zip_directory(directory_path, zip_file_name):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_file_name, 'w') as zipf:
        # Iterate over all the files in the directory
        for root, _, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Add the file to the zip file
                zipf.write(file_path, os.path.relpath(file_path, directory_path))

    print(f"{zip_file_name} has been created.")

# Specify the directory path and the name of the zip file
directory_path = "../data2"
zip_file_name = "../compressed.zip"

# Call the function to compress the directory
zip_directory(directory_path, zip_file_name)
